<a href="https://colab.research.google.com/github/Aravindkumar-Rajendran/spell-correction-model/blob/main/spell_check_and_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

Datasets available for spell correction task are very few.

https://paperswithcode.com/task/spelling-correction#datasets  
https://www.kaggle.com/datasets/bittlingmayer/spelling?select=wikipedia.txt  
https://github.com/neuspell/neuspell/tree/master/data/traintest  



## Exploring Github typo corpus dataset

https://github.com/mhagiwara/github-typo-corpus

Dataset download url: https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz


In [4]:
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz

--2022-10-19 16:05:09--  https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz
Resolving github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)... 54.231.130.17
Connecting to github-typo-corpus.s3.amazonaws.com (github-typo-corpus.s3.amazonaws.com)|54.231.130.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 43769081 (42M) [application/x-gzip]
Saving to: ‘github-typo-corpus.v1.0.0.jsonl.gz’

github-typo-corpus. 100%[===================>]  41.74M  29.2MB/s    in 1.4s    

2022-10-19 16:05:11 (29.2 MB/s) - ‘github-typo-corpus.v1.0.0.jsonl.gz’ saved [43769081/43769081]



In [5]:
!ls

github-typo-corpus.v1.0.0.jsonl.gz  sample_data


In [6]:
!gzip -d github-typo-corpus.v1.0.0.jsonl.gz

In [7]:
import pandas as pd

typo_data = pd.read_json("github-typo-corpus.v1.0.0.jsonl", lines=True)

In [8]:
typo_data.head()

,repo,commit,message,edits
0,https://github.com/abacritt/angularx-social-login,d4c912f5ccd70c81f424fadbf1fe1a2ecb942f07,Fix typo\n,[{'src': {'text': ' IN.User.authori...
1,https://github.com/abacritt/angularx-social-login,8beb5a5ebee0882142541dc84c004f6ce3165f94,fix typo\n\nfix typo in firstname\n,[{'src': {'text': ' user.em...
2,https://github.com/abahmed/Deer,44781b8842c7e647d1f5d2417d21244e815c5eec,fixed typo (#263)\n\n,[{'src': {'text': ' :de: | Deutsch...
3,https://github.com/abakan/ablog,1cee106975e3137cb9a667729e832cc9494f0692,Fixed typo.\n,"[{'src': {'text': ' :nocoments:', 'path..."
4,https://github.com/abakan/ablog,4e11caf1f7ebe611ffb08f8a6909ac6752d784cd,Fixed typo\n,[{'src': {'text': ' You can disable comments...


In [9]:
import random
idx = random.randint(0, len(typo_data))
typo_data['edits'][idx]

[{'src': {'text': 'If you\'re unable to use promises or other supported async objects, you may enable "callback mode" by defining your test with `test.cb([title\', fn)`. Tests declared this way **must** be manually ended with `t.end()`. This mode is mainly intended for testing callback-style APIs.',
   'path': 'readme.md',
   'lang': 'eng',
   'ppl': 5.8147712336705375},
  'tgt': {'text': 'If you\'re unable to use promises or other supported async objects, you may enable "callback mode" by defining your test with `test.cb([title], fn)`. Tests declared this way **must** be manually ended with `t.end()`. This mode is mainly intended for testing callback-style APIs.',
   'path': 'readme.md',
   'lang': 'eng',
   'ppl': 5.74987434397607},
  'prob_typo': 0.9211678380303181,
  'is_typo': True}]

In [10]:
edits_list = typo_data['edits'].to_list()

1. Ignoring URLs
2. Ignoring words inside brackets
3. Ignoring non-english words

In [67]:
import re
import nltk
nltk.download('words')


vocab = []
word_matches = {}

# Regex to check URL
pattern = ("((http|https)://)(www.)?" +
            "[a-zA-Z0-9@:%._\\+~#?&//=]" +
            "{2,256}\\.[a-z]" +
            "{2,6}\\b([-a-zA-Z0-9@:%" +
            "._\\+~#?&//=]*)")
     
alphabet = 'abcdefghijklmnopqrstuvwxyz'

for edit in edits_list:
    for text in edit:
        tgt_sentence = text['tgt']['text'].lower()
        src_sentence = text['src']['text'].lower()
        src_words = tgt_words = []
        for word in nltk.wordpunct_tokenize(tgt_sentence):
            if word:
                if re.search(pattern, word):
                    break
                # print(word)
                chars = [chr for chr in word if (chr in alphabet) or \
                            chr in [" ", "_"]]
                # print(chars)
                word = "".join(chars)
                if word and len(word) > 1:
                    tgt_words.append(word)
                    if "_" not in word: 
                        vocab.append(word)

        for word in nltk.wordpunct_tokenize(src_sentence):
            if word:                
                if re.search(pattern, word):
                    break
                chars = [chr for chr in word if (chr in alphabet) or \
                         chr in [" ", "_"]]
                word = "".join(chars)
                if word and len(word) > 1:
                    src_words.append(word)
        word_list = [{tgt:src} for tgt, src in zip(tgt_words, src_words)]
        if word_list:
            for match in word_list:
                word_matches.update(match)
        

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [68]:
len(vocab)

4492012

In [59]:
# removing the duplicates
# vocab = list(set(vocab))

In [60]:
len(vocab)

108969

In [61]:
ind = random.randint(0, len(vocab) - 100)
vocab[ind:ind + 100]

['kucaklar',
 'datatabletype',
 'todoservice',
 'successes',
 'ngdoc',
 'arizona',
 'streampipelinepersistentlogs',
 'wikiproject',
 'deviot',
 'gltige',
 'assertions',
 'containerconfig',
 'dua',
 'fragmentation',
 'scpbus',
 'graphdef',
 'template',
 'structures',
 'marston',
 'aggravate',
 'ki',
 'logon',
 'isarray',
 'buildsystem',
 'minimal',
 'fling',
 'selecthero',
 'influences',
 'harmonics',
 'kullanlabilir',
 'vconnection',
 'simonboulier',
 'serviceaccountcredentials',
 'nofork',
 'amazonsfullaccess',
 'gollum',
 'trfk',
 'chance',
 'sebastienros',
 'recognizing',
 'solucionar',
 'extractimagefilter',
 'chicagoboss',
 'reactquill',
 'horizontallayout',
 'propagating',
 'cranelift',
 'stackalloc',
 'controlplane',
 'gtie',
 'outcolor',
 'braintreefragment',
 'clv',
 'framedata',
 'demoed',
 'tokyo',
 'vuejs',
 'hateoas',
 'purely',
 'note',
 'fazem',
 'multicasting',
 'packageservice',
 'metatada',
 'stimodeltest',
 'featuredjobitem',
 'vrei',
 'markup',
 'gluonts',
 'rpsyste

In [69]:
with open("github_typo_vocab.txt", "w") as f:
    for v in vocab:
        f.write(v)
        f.write("\n")

#### Observation
Dataset had urls, non-english words and others noises. Removed in preprocessing.

## MSFT spell correction dataset

https://www.microsoft.com/en-us/download/details.aspx?id=52418  
https://download.microsoft.com/download/B/8/E/B8E6EF6B-7D0B-456C-A774-D9E454765EFC/MSR%20Spelling%20Correction%20Data.zip

In [ ]:
!wget https://download.microsoft.com/download/B/8/E/B8E6EF6B-7D0B-456C-A774-D9E454765EFC/MSR%20Spelling%20Correction%20Data.zip

--2022-10-19 05:50:41--  https://download.microsoft.com/download/B/8/E/B8E6EF6B-7D0B-456C-A774-D9E454765EFC/MSR%20Spelling%20Correction%20Data.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.60.85.2, 2600:1408:9000:6ac::317f, 2600:1408:9000:68a::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.60.85.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203875 (199K) [application/octet-stream]
Saving to: ‘MSR Spelling Correction Data.zip’

MSR Spelling Correc 100%[===================>] 199.10K  1.03MB/s    in 0.2s    

2022-10-19 05:50:42 (1.03 MB/s) - ‘MSR Spelling Correction Data.zip’ saved [203875/203875]



In [ ]:
!unzip 'MSR Spelling Correction Data.zip'

Archive:  MSR Spelling Correction Data.zip
  inflating: MSR Spelling Correction Data.rtf  
  inflating: ja_keystroke_pairs.sorted.txt  
  inflating: en_keystroke_pairs.sorted.txt  
  inflating: README.txt              


In [ ]:
!ls

 big.txt			  'MSR Spelling Correction Data.rtf'
 en_keystroke_pairs.sorted.txt	  'MSR Spelling Correction Data.zip'
 github-typo-corpus.v1.0.0.jsonl   README.txt
 ja_keystroke_pairs.sorted.txt	   sample_data


# Traditional Approach based spell correction


## Peter Norvig's algorithm

**Operations involved**

1. Finding mispelled words based on a vocabulary of correct words
2. Getting the relevant words based on edit distance (INSERT, DELETE, SWAP, REPLACE)
3. Removing the words that are not n vocabulary and creating candidates.
4. Choosing most likely candidate based on probability score.   
**Probability p(w) = word count c(w) / total no. of words in the corpus t**

In [ ]:
!wget https://norvig.com/big.txt

--2022-10-19 04:54:26--  https://norvig.com/big.txt
Resolving norvig.com (norvig.com)... 158.106.138.13
Connecting to norvig.com (norvig.com)|158.106.138.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6488666 (6.2M) [text/plain]
Saving to: ‘big.txt’

big.txt             100%[===================>]   6.19M  18.3MB/s    in 0.3s    

2022-10-19 04:54:27 (18.3 MB/s) - ‘big.txt’ saved [6488666/6488666]



In [ ]:
with open("big.txt", 'r') as f:
    data = f.read()

In [ ]:
data[:1000]

'The Project Gutenberg EBook of The Adventures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in our series by Sir Arthur Conan Doyle)\n\nCopyright laws are changing all over the world. Be sure to check the\ncopyright laws for your country before downloading or redistributing\nthis or any other Project Gutenberg eBook.\n\nThis header should be the first thing seen when viewing this Project\nGutenberg file.  Please do not remove it.  Do not change or edit the\nheader without written permission.\n\nPlease read the "legal small print," and other information about the\neBook and Project Gutenberg at the bottom of this file.  Included is\nimportant information about your specific rights and restrictions in\nhow the file may be used.  You can also find out about how to make a\ndonation to Project Gutenberg, and how to get involved.\n\n\n**Welcome To The World of Free Plain Vanilla Electronic Texts**\n\n**eBooks Readable By Both Humans and By Computers, Since 1971**\n\n*****These eBooks 



---
⚡ Below code is taken from Peter Norvig's Algorithm for spelling correction.

---








In [ ]:
"""Spelling Corrector in Python 3; see http://norvig.com/spell-correct.html

Copyright (c) 2007-2016 Peter Norvig
MIT license: www.opensource.org/licenses/mit-license.php
"""

# Spelling Corrector 

import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    c = candidates(word)
    return max(c, key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [ ]:
# Testing 

# def edits1(word):
#     "All edits that are one edit away from `word`."
#     letters    = 'abcdefghijklmnopqrstuvwxyz'
#     splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
#     print("splits: ", splits)
#     deletes    = [L + R[1:]               for L, R in splits if R]
#     print("deletes: ", deletes)
#     transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
#     print("transposes: ", transposes)
#     replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
#     print("replaces: ", replaces)
#     inserts    = [L + c + R               for L, R in splits for c in letters]
#     print("inserts: ", inserts)

#     return set(deletes + transposes + replaces + inserts)
    
# edits1('spling')

In [ ]:
[correction(word) for word in '"connect (host, port, anchor_amount): opens a channel with another eclair or lightningd instance", '.split()]

['connect',
 'host',
 'ports',
 'anchor_amount):',
 'opens',
 'a',
 'channel',
 'with',
 'another',
 'clair',
 'or',
 'lightning',
 'instance']

In [ ]:
sentence = "Quite a few people went ahead and they bought their owninfrastructure and now they rent the services to other people, and when you talk about this infrastructurethe, quite a few people out there who are actually providingthese cloud services to different peopleacross the globe."
corrected_words = []
for word in sentence.split():
    corrected_words.append(correction(word))
print(" ".join(corrected_words))

quite a few people went ahead and they bought their owninfrastructure and now they rent the services to other people and when you talk about this infrastructurethe, quite a few people out there who are actually providingthese cloud services to different peopleacross the globe


### Observation
Each singe word will be evaluated and word segmentation for example "owninfrastruture" is not spell corrected as candidates available. And it is slow as it looks for all the edits in a word.  

## Peter-Norvig - Using custom vocab created from github typo corpus

In [70]:
"""Spelling Corrector in Python 3; see http://norvig.com/spell-correct.html

Copyright (c) 2007-2016 Peter Norvig
MIT license: www.opensource.org/licenses/mit-license.php
"""

# Spelling Corrector 

import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('github_typo_vocab.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    c = candidates(word)
    return max(c, key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


In [83]:
idx = random.randint(0, len(edits_list)-1)

for edit in edits_list[idx:idx+1]:
    for text in edit:
        src_sentence = text['src']['text'].lower()
src_sentence

'the `servicebroker` has a new `fatal` method. if you call it, you can log message with `fatal` level and exit the process with code `2`.'

In [84]:
import nltk

sentence = src_sentence
corrected_words = []
for word in nltk.wordpunct_tokenize(sentence.strip()):
    corrected_words.append(correction(word))
print(" ".join(corrected_words))

the to servicebroker to has as new to fatal to method to if you call it to you can log message with to fatal to level and exit the process with code to to to


## Symspell approach

In [ ]:
!pip install symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 12.9 MB/s 
     |████████████████████████████████| 125 kB 52.2 MB/s 


In [ ]:
import pkg_resources
from symspellpy import SymSpell

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
bigram_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_bigramdictionary_en_243_342.txt"
)
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

True

In [ ]:
text = "Quite a fw people 45 went ahead and they buought their owninfrastructure and now they rent the services to other people, and when you talk about this infrastructurethe, quite a few people out there who are actually providingthese cloud services to different peopleacross the globe."
suggestions = sym_spell.lookup_compound(text, max_edit_distance=2, 
                                        ignore_non_words=True, transfer_casing=True)
for suggestion in suggestions:
    print(suggestion)

Quite a few people 45 went ahead and they brought their own infrastructure and now they rent the services to other people and when you talk about this infrastructure the quite a few people out there who are actually providing these cloud services to different people across the globe, 9, 0


In [ ]:
print(suggestion.term)

Quite a few people 45 went ahead and they brought their own infrastructure and now they rent the services to other people and when you talk about this infrastructure the quite a few people out there who are actually providing these cloud services to different people across the globe


### Observation
Word segmentation is also possible with this approach of using n-gram words as a dictionary to lookup and verify.